# Lab 6 - Microsoft Azure

In [1]:
import requests, json
import pandas as pd
from azure.cognitiveservices.search.newssearch import NewsSearchAPI
from msrest.authentication import CognitiveServicesCredentials
subscription_key = 'INSERT KEY'
search_term = ['Game of Thrones']

client = NewsSearchAPI(CognitiveServicesCredentials(subscription_key))

newssearch = []
newstext = []
newssource = []
location = []
colors = []

def BizLoc(name):
    place = 'https://atlas.microsoft.com/search/poi/json?subscription-key=INSERT KEY&api-version=1.0&query='+str(name)+'&limit=1'
    response = requests.get(place)
    stuff = response.json()
    if stuff['results']:
        x, y = stuff['results'][0]['position']['lon'], stuff['results'][0]['position']['lat']
    else:
        x, y = 0,0
    return x,y

def sentiment(text):
    import requests, json
    documents = {'documents': [
        {'id': 1, 'text': text}
    ]}
    azure_key = 'INSERT KEY'
    azure_endpoint = 'https://westus.api.cognitive.microsoft.com/text/analytics/v2.0'
    assert azure_key
    sentiment_azure = azure_endpoint + '/sentiment'
    
    headers = {'Ocp-Apim-Subscription-Key' : azure_key}
    response = requests.post(sentiment_azure, headers=headers, json=documents)
    sentiments = response.json()
    return sentiments

#you can collapse result (don't need it) and you need to append to azure_score within your iteration
azure_results = [sentiment(text) for text in newstext]
azure_score = [row['documents'][0]['score'] for row in azure_results]

n = 0
while n < 100:
    news_result = client.news.search(query=search_term, market="en-us", count=10, offset = n)
    if news_result.value:
        for news in news_result.value:
            newstext.append(str(news.description))
            newssource.append(str(news.provider[0].name))
            #you aren't geocoding newssource... you're geocoding... news
            location.append(BizLoc(news.provider[0].name))
            for i in range(0, len (news_result.value)):
                if azure_score[i] >= .75:
                    colors.append('lightgreen')
                elif azure_score[i] > .5:
                    colors.append('green')
                elif azure_score[i] <= .25:
                    colors.append('darkblue')
                elif azure_score[i] < .5:
                    colors.append('blue')
                elif azure_score[i] == .5:
                    colors.append('gray')
                else:
                    colors.append('white')
    n = n + 10
    #newssearch.append(client.news.search)

#Kept for future use    
'''print(newstext)
else:
print('No news data..')'''

azure = list(zip(newssource, location, newstext, azure_score, colors))
columns = ['source', 'location', 'text', 'score', 'colors']
azure_df = pd.DataFrame(azure, columns = columns)

azure_df

,source,location,text,score,colors


In [5]:
import shapely, shapely.geometry, fiona, fiona.crs, pandas as pd, folium, geopandas
from folium.plugins import MarkerCluster

coords = zip(azure_df['location'])

geometry = [shapely.geometry.Point(c) for c in coords]

geolist = geopandas.GeoSeries(geometry)

geoPoints = geopandas.GeoDataFrame(
        azure_df,
        geometry=geolist)

geoPoints.crs=fiona.crs.from_epsg(4236)

map = folium.Map(
                location = [33, -23],
                zoom_start = 2,
                tiles ='Stamen Toner')

cluster = MarkerCluster()

for i in range(0, len(azure_df)):
    cluster.add_child(folium.Marker(location = [azure_df.iloc[i]['location'][1], azure_df.iloc[i]['location'][0]]
                      , popup = folium.Popup(azure_df.iloc[i]['source']), icon = folium.Icon(icon = 'comment', color = azure_df.iloc[i]['colors'])))

map.add_child(cluster)

map

In [8]:
import shapely, shapely.geometry, fiona, fiona.crs, pandas as pd, folium, geopandas
from folium.plugins import HeatMap

coords = zip(azure_df['location'])

geometry = [shapely.geometry.Point(c) for c in coords]

geolist = geopandas.GeoSeries(geometry)

geoPoints = geopandas.GeoDataFrame(
        azure_df,
        geometry=geolist)

geoPoints.crs=fiona.crs.from_epsg(4236)

mapa = folium.Map(
                location = [33, -23],
                zoom_start = 2,
                tiles ='Stamen Terrain')

# convert to (n, 2) nd-array format for heatmap
mapaheat = azure_df['location'].values

# plot heatmap
mapa.add_child(folium.plugins.HeatMap(mapaheat, radius=15))

#points
for index, row in azure_df.iterrows():
    folium.CircleMarker([row[azure_df.iloc[i]['location'][1], azure_df.iloc[i]['location'][0]]],
                        radius = 2.5,
                        popup = row['source'],
                        fill_color = '#3db7e4',
                        fill_opacity = '0.1',
                        opacity = '0.2',
                        weight = '2'
                       ).add_to(mapa)
    
mapa